In [21]:
## Pacotes necessários:
#install.packages("e1071") 
#install.packages("caret")
#install.packages("randomForest")
library("caret")
library("randomForest")

In [22]:
#carregamento dos dados

dados <- read.csv("databases/1 - Cancer de Mama - Dados.csv")

dados$Id <- NULL
head(dados)

,Cl.thickness,Cell.size,Cell.shape,Marg.adhesion,Epith.c.size,Bare.nuclei,Bl.cromatin,Normal.nucleoli,Mitoses,Class
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
1,5,1,1,1,2,1,3,1,1,benign
2,5,4,4,5,7,10,3,2,1,benign
3,3,1,1,1,2,2,3,1,1,benign
4,6,8,8,1,3,4,3,7,1,benign
5,4,1,1,3,2,1,3,1,1,benign
6,8,10,10,8,7,10,9,7,1,malignant


In [23]:
#separação em base de teste e treino
set.seed(2034)
indices <- createDataPartition(dados$Class, p = 0.8, list = FALSE)

treino <- dados[indices, ]
teste <- dados[-indices, ]

In [24]:
# treinando o modelo
set.seed(2034)
rf <- train(Class ~ ., data = treino, method = "rf")
rf

Random Forest 

560 samples
  9 predictor
  2 classes: 'benign', 'malignant' 

No pre-processing
Resampling: Bootstrapped (25 reps) 
Summary of sample sizes: 560, 560, 560, 560, 560, 560, ... 
Resampling results across tuning parameters:

  mtry  Accuracy   Kappa    
  2     0.9652311  0.9227391
  5     0.9600717  0.9107838
  9     0.9548117  0.8990943

Accuracy was used to select the optimal model using the largest value.
The final value used for the model was mtry = 2.

In [25]:
# aplicando na base de teste
predict_rf <- predict(rf, teste)
confusionMatrix(predict_rf, as.factor(teste$Class))

Confusion Matrix and Statistics

           Reference
Prediction  benign malignant
  benign        87         2
  malignant      4        46
                                         
               Accuracy : 0.9568         
                 95% CI : (0.9084, 0.984)
    No Information Rate : 0.6547         
    P-Value [Acc > NIR] : <2e-16         
                                         
                  Kappa : 0.9055         
                                         
 Mcnemar's Test P-Value : 0.6831         
                                         
            Sensitivity : 0.9560         
            Specificity : 0.9583         
         Pos Pred Value : 0.9775         
         Neg Pred Value : 0.9200         
             Prevalence : 0.6547         
         Detection Rate : 0.6259         
   Detection Prevalence : 0.6403         
      Balanced Accuracy : 0.9572         
                                         
       'Positive' Class : benign         
                   

In [26]:
#aplicando cross-validation
ctrl <- trainControl(method = "cv", number = 10)

set.seed(2034)
rf <- train(Class ~ ., data =  treino, method = "rf", trControl = ctrl)
rf

#experimentando na base de teste
predict_rf <- predict(rf, teste)
confusionMatrix(predict_rf, as.factor(teste$Class))

Random Forest 

560 samples
  9 predictor
  2 classes: 'benign', 'malignant' 

No pre-processing
Resampling: Cross-Validated (10 fold) 
Summary of sample sizes: 504, 504, 504, 504, 503, 505, ... 
Resampling results across tuning parameters:

  mtry  Accuracy   Kappa    
  2     0.9696104  0.9332907
  5     0.9624675  0.9168661
  9     0.9607131  0.9129578

Accuracy was used to select the optimal model using the largest value.
The final value used for the model was mtry = 2.

Confusion Matrix and Statistics

           Reference
Prediction  benign malignant
  benign        87         2
  malignant      4        46
                                         
               Accuracy : 0.9568         
                 95% CI : (0.9084, 0.984)
    No Information Rate : 0.6547         
    P-Value [Acc > NIR] : <2e-16         
                                         
                  Kappa : 0.9055         
                                         
 Mcnemar's Test P-Value : 0.6831         
                                         
            Sensitivity : 0.9560         
            Specificity : 0.9583         
         Pos Pred Value : 0.9775         
         Neg Pred Value : 0.9200         
             Prevalence : 0.6547         
         Detection Rate : 0.6259         
   Detection Prevalence : 0.6403         
      Balanced Accuracy : 0.9572         
                                         
       'Positive' Class : benign         
                   

In [27]:
#experimentando com outros valores de mtry
tune_grid <- expand.grid(mtry = c(2, 5, 7, 9))

set.seed(2034)
rf <- train(Class ~ ., data = treino, method = "rf", trControl = ctrl, tuneGrid = tune_grid)
rf

#experimentando novo modelo na base de teste
predict_rf <- predict(rf, teste)
confusionMatrix(predict_rf, as.factor(teste$Class))

Random Forest 

560 samples
  9 predictor
  2 classes: 'benign', 'malignant' 

No pre-processing
Resampling: Cross-Validated (10 fold) 
Summary of sample sizes: 504, 504, 504, 504, 503, 505, ... 
Resampling results across tuning parameters:

  mtry  Accuracy   Kappa    
  2     0.9713961  0.9373253
  5     0.9678247  0.9290894
  7     0.9624989  0.9168904
  9     0.9642846  0.9209236

Accuracy was used to select the optimal model using the largest value.
The final value used for the model was mtry = 2.

Confusion Matrix and Statistics

           Reference
Prediction  benign malignant
  benign        87         1
  malignant      4        47
                                          
               Accuracy : 0.964           
                 95% CI : (0.9181, 0.9882)
    No Information Rate : 0.6547          
    P-Value [Acc > NIR] : <2e-16          
                                          
                  Kappa : 0.9216          
                                          
 Mcnemar's Test P-Value : 0.3711          
                                          
            Sensitivity : 0.9560          
            Specificity : 0.9792          
         Pos Pred Value : 0.9886          
         Neg Pred Value : 0.9216          
             Prevalence : 0.6547          
         Detection Rate : 0.6259          
   Detection Prevalence : 0.6331          
      Balanced Accuracy : 0.9676          
                                          
       'Positive' Class : benign          

In [28]:
# predicao de novos casos
dados_novos_casos <- read.csv("databases/1 - Cancer de Mama - Dados - Novos Casos.csv")

dados_novos_casos$Id <- NULL
dados_novos_casos$Class <- NULL
View(dados_novos_casos)
predict_rf <- predict(rf, dados_novos_casos)
resultado <- cbind(dados_novos_casos, predict_rf)
View(resultado)

Cl.thickness,Cell.size,Cell.shape,Marg.adhesion,Epith.c.size,Bare.nuclei,Bl.cromatin,Normal.nucleoli,Mitoses
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
2,1,5,1,7,1,3,10,5
2,3,4,5,3,1,3,2,1
7,1,1,5,2,2,2,1,1


Cl.thickness,Cell.size,Cell.shape,Marg.adhesion,Epith.c.size,Bare.nuclei,Bl.cromatin,Normal.nucleoli,Mitoses,predict_rf
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
2,1,5,1,7,1,3,10,5,benign
2,3,4,5,3,1,3,2,1,benign
7,1,1,5,2,2,2,1,1,benign
